In [24]:
import warnings

import pandas as pd

from jre_utils.datapath import (
    factor_data_paths,
    model_ready_data_paths,
    get_derived_csv_path,
    get_derived_lpa_path,
    get_derived_plps_path,
)

from jre_utils.process import (
    get_most_active_municipalities
)

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

In [25]:
asset_type = "building"

metrics = {
    "weighted_mean": "unit_price_wmean",
    "weighted_median": "unit_price_wmedian",
    "mean": "unit_price_mean",
    "median": "unit_price_median",
}

dataset_paths = {
    "transactions": get_derived_csv_path(asset_type),
    "lpa": get_derived_lpa_path(),
    "plps": get_derived_plps_path()
}

granularity_columns = ["area_code", "area"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

metric_key = "weighted_median"
metric = metrics[metric_key]
metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric
upcoming_metric_pct_chg = "upcoming_" + metric_pct_chg


In [26]:
dataset_key = "transactions"
core_path = dataset_paths[dataset_key]
population_path = factor_data_paths["processed"]["population"]["municipality"]
migration_path = factor_data_paths["processed"]["migration"]["municipality"]
taxable_income_path = factor_data_paths["processed"]["taxable_income"]["municipality"]
new_dwellings_path = factor_data_paths["processed"]["new_dwellings"]["municipality"]
lfs_revenue_path = factor_data_paths["processed"]["lfs_revenue_breakdown"]["municipality"]


In [27]:
df = pd.read_csv(core_path)

population_df = pd.read_csv(population_path)
migration_df = pd.read_csv(migration_path)
taxable_income_df = pd.read_csv(taxable_income_path)
new_dwellings_df = pd.read_csv(new_dwellings_path)
lfs_revenue_df= pd.read_csv(lfs_revenue_path)

df = (
    df.merge(population_df, on=group_by_columns, how="left")
    .merge(migration_df, on=group_by_columns, how="left")
    .merge(taxable_income_df, on=group_by_columns, how="left")
    .merge(new_dwellings_df, on=group_by_columns, how="left")
    .merge(lfs_revenue_df, on=group_by_columns, how="left")
)

In [28]:
df["migrations_is_available"] = df["net_migration_ratio"].notnull().astype(int)
df["taxable_income_is_available"] = df["taxable_income"].notnull().astype(int)
df["total_tax_is_available"] = df["total_tax"].notnull().astype(int)

# Might go back and undo the new dwellings filling for unknown municipalities.
df["dwellings_is_available"] = df["new_dwellings"].notnull().astype(int)

df = df.fillna(0)

In [29]:
df

,year,area_code,area,unit_price_wmean,unit_price_wmedian,unit_price_mean,unit_price_median,total_traded_area,count,population,net_migration_ratio,taxable_income,taxpayer_count,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,new_dwellings,existing_dwellings,new_dwellings_ratio,total_tax,total_tax_growth,migrations_is_available,taxable_income_is_available,total_tax_is_available,dwellings_is_available
0,2023,40100,Fukuoka-ken Kitakyushu-shi,111020.840355,75850.000000,112618.534187,77171.215881,89010.0,278.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0
1,2023,40130,Fukuoka-ken Fukuoka-shi,383051.876669,260000.000000,417016.083864,283870.967742,95515.0,365.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0
2,2023,40202,Fukuoka-ken Omuta-shi,27522.713507,10515.650080,44566.062666,15381.818182,8255.0,26.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0
3,2023,40203,Fukuoka-ken Kurume-shi,99893.243862,57081.815363,110045.308156,91304.347826,32785.0,99.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0
4,2023,40204,Fukuoka-ken Nogata-shi,35340.751043,26605.692444,46028.433096,29411.764706,7190.0,25.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27723,2006,29205,Nara-ken Kashihara-shi,292307.692308,292307.692308,292307.692308,292307.692308,260.0,2.0,127413.0,-0.006703,169637011.0,48439.0,3502.075002,0.024555,-0.030735,1025.0,49344.0,0.020773,15056805.0,0.020687,1,1,1,1
27724,2006,29207,Nara-ken Gojo-shi,44357.976654,44357.976654,45570.231959,45570.231959,2570.0,2.0,32653.0,-0.012924,38918213.0,12670.0,3071.682163,0.001536,-0.051189,107.0,13273.0,0.008061,3505676.0,-0.013917,1,1,1,1
27725,2006,29209,Nara-ken Ikoma-shi,87272.727273,87272.727273,87272.727273,87272.727273,110.0,1.0,115538.0,0.002839,217154351.0,49240.0,4410.120857,0.044310,-0.015477,1237.0,42925.0,0.028818,16419633.0,0.016372,1,1,1,1
27726,2006,29212,Nara-ken Uda-shi,16216.216216,16216.216216,16216.216216,16216.216216,185.0,1.0,32836.0,-0.011207,44256793.0,13620.0,3249.397430,0.022078,-0.044485,78.0,12349.0,0.006316,3023683.0,0.001659,1,1,1,1


In [30]:
# prepare metrics
df = df.sort_values(by=group_by_columns, ascending=True)
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change()

# set up target variables
df[upcoming_metric_pct_chg] = df.groupby(granularity_columns)[metric_pct_chg].shift(-1)

# time box
# start_year = 2005
# end_year = 2023

# df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

In [31]:
# most frequent municipalities
# skip this and build custom cost function using count as parameter. The lower the count, the lower the cost.

# df = get_most_active_municipalities(df, n=1500)
# df = df[df["count"] > 5]

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 27728 entries, 24017 to 3378
Data columns (total 27 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   year                                 27728 non-null  int64  
 1   area_code                            27728 non-null  int64  
 2   area                                 27728 non-null  object 
 3   unit_price_wmean                     27728 non-null  float64
 4   unit_price_wmedian                   27728 non-null  float64
 5   unit_price_mean                      27728 non-null  float64
 6   unit_price_median                    27728 non-null  float64
 7   total_traded_area                    27728 non-null  float64
 8   count                                27728 non-null  float64
 9   population                           27728 non-null  float64
 10  net_migration_ratio                  27728 non-null  float64
 11  taxable_income                

In [33]:
df.columns

Index(['year', 'area_code', 'area', 'unit_price_wmean', 'unit_price_wmedian',
       'unit_price_mean', 'unit_price_median', 'total_traded_area', 'count',
       'population', 'net_migration_ratio', 'taxable_income', 'taxpayer_count',
       'taxable_income_per_taxpayer', 'taxable_income_growth',
       'taxable_income_per_taxpayer_growth', 'new_dwellings',
       'existing_dwellings', 'new_dwellings_ratio', 'total_tax',
       'total_tax_growth', 'migrations_is_available',
       'taxable_income_is_available', 'total_tax_is_available',
       'dwellings_is_available', 'unit_price_wmedian_pct_chg',
       'upcoming_unit_price_wmedian_pct_chg'],
      dtype='object')

In [34]:
columns = [
    metric_pct_chg,
    metric,
    "year",
    "count",
    "total_traded_area",
    "population",
    "taxpayer_count",
    "taxable_income",
    "taxable_income_per_taxpayer",
    "taxable_income_growth",
    "taxable_income_per_taxpayer_growth",
    "total_tax",
    "total_tax_growth",
    "new_dwellings",
    "existing_dwellings",
    "net_migration_ratio",
    "new_dwellings_ratio",
    "migrations_is_available",
    "taxable_income_is_available",
    "dwellings_is_available",
    "total_tax_is_available",
    "area_code",  # id
]

# add ratios and growths if necessary

df = df[columns]
df = df.dropna()

In [35]:
print(f"Initial Size: ", df.shape[0])
q = 0.01
filtered_df = df.copy()
filter_col = metric_pct_chg
filtered_df = filtered_df[
    (filtered_df[filter_col] >= filtered_df[filter_col].quantile(q))
    & (filtered_df[filter_col] <= filtered_df[filter_col].quantile(1 - q))
]
print(f"Filtered Size: ", filtered_df.shape[0])
filtered_df.describe()

Initial Size:  25992
Filtered Size:  25472


,unit_price_wmedian_pct_chg,unit_price_wmedian,year,count,total_traded_area,population,taxpayer_count,taxable_income,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,total_tax,total_tax_growth,new_dwellings,existing_dwellings,net_migration_ratio,new_dwellings_ratio,migrations_is_available,taxable_income_is_available,dwellings_is_available,total_tax_is_available,area_code
count,25472.000000,2.547200e+04,25472.000000,25472.000000,25472.000000,2.547200e+04,2.547200e+04,2.547200e+04,25472.000000,25472.000000,25472.000000,2.547200e+04,25472.000000,25472.000000,2.547200e+04,25472.000000,25472.000000,25472.000000,25472.000000,25472.000000,25472.000000,25472.000000
mean,0.225440,6.835178e+04,2015.450612,70.856548,18836.150086,7.603160e+04,3.252745e+04,1.091318e+08,2517.451462,0.002088,0.001972,1.185357e+07,0.003805,497.025754,3.120507e+04,-0.003564,0.006021,0.933810,0.876963,0.451319,0.943389,21334.462312
std,1.013658,1.868260e+05,4.601773,189.407582,38089.595844,1.968948e+05,8.870744e+04,3.329141e+08,1078.379123,0.043107,0.032731,3.870515e+07,0.063054,1773.254926,9.692370e+04,0.007663,0.008067,0.248619,0.328486,0.497634,0.231103,14037.327508
min,-0.901671,7.969444e+01,2006.000000,1.000000,60.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,-0.682739,-0.694693,0.000000e+00,-0.677251,0.000000,0.000000e+00,-0.313065,0.000000,0.000000,0.000000,0.000000,0.000000,1100.000000
25%,-0.289994,1.186053e+04,2012.000000,6.000000,2960.000000,8.174500e+03,2.688000e+03,7.159464e+06,2438.675588,-0.009826,-0.007155,1.016966e+06,-0.015249,0.000000,0.000000e+00,-0.007151,0.000000,1.000000,1.000000,0.000000,1.000000,9342.000000
50%,-0.011916,2.561702e+04,2015.000000,20.000000,8090.000000,2.543850e+04,9.661000e+03,2.616406e+07,2705.771274,0.000690,0.000000,3.192085e+06,0.000000,0.000000,0.000000e+00,-0.003158,0.000000,1.000000,1.000000,0.000000,1.000000,20439.000000
75%,0.349895,6.435983e+04,2019.000000,62.000000,19378.750000,6.577250e+04,2.709700e+04,7.916949e+07,3015.250201,0.018158,0.011314,9.189060e+06,0.017930,341.000000,2.682250e+04,0.000000,0.011740,1.000000,1.000000,1.000000,1.000000,33100.000000
max,7.933118,6.453629e+06,2023.000000,4358.000000,666150.000000,3.811873e+06,1.906224e+06,7.965148e+09,12667.020000,1.066040,1.026488,8.672765e+08,4.759148,41746.000000,1.916062e+06,0.144218,0.120270,1.000000,1.000000,1.000000,1.000000,47382.000000


In [36]:
filtered_df.to_csv(model_ready_data_paths[f"sequence_{dataset_key}_{metric_key}"], index=False)